In [30]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors



In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip
books = pd.read_csv("books.csv")  # This contains book information (book_id, title, etc.)
ratings = pd.read_csv("ratings.csv")  # This contains user ratings (user_id, book_id, rating)


--2025-03-31 17:02:44--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  57.9MB/s    in 0.4s    

2025-03-31 17:02:44 (57.9 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Filter users with more than 200 ratings
user_rating_count = ratings['user_id'].value_counts()
valid_users = user_rating_count[user_rating_count >= 200].index
ratings_filtered = ratings[ratings['user_id'].isin(valid_users)]

# Filter books with more than 100 ratings
book_rating_count = ratings_filtered['book_id'].value_counts()
valid_books = book_rating_count[book_rating_count >= 100].index
ratings_filtered = ratings_filtered[ratings_filtered['book_id'].isin(valid_books)]

# Create a user-item matrix (user_id vs book_id)
user_book_matrix = ratings_filtered.pivot_table(index='user_id', columns='book_id', values='rating', fill_value=0)

# Normalize the matrix if needed
user_book_matrix_normalized = user_book_matrix - user_book_matrix.mean(axis=1).values.reshape(-1, 1)


In [ ]:
# Fit the KNN model
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
model.fit(user_book_matrix_normalized)

# Create a function to get recommendations based on book title
def get_recommends(book_title):
    # Find the book_id corresponding to the title
    book_id = books[books['title'] == book_title].iloc[0]['book_id']

    # Get the index of the book_id in the user_book_matrix
    book_idx = user_book_matrix.columns.get_loc(book_id)

    # Find the nearest neighbors (6 because the first one is the book itself)
    distances, indices = model.kneighbors(user_book_matrix.iloc[:, book_idx].values.reshape(1, -1), n_neighbors=6)

    # Prepare the output
    recommendations = []
    for i in range(1, len(indices[0])):  # Skip the first book (it will be the same book)
        recommended_book_id = user_book_matrix.columns[indices[0][i]]
        recommended_book_title = books[books['book_id'] == recommended_book_id].iloc[0]['title']
        recommendations.append([recommended_book_title, distances[0][i]])

    return [book_title, recommendations]


In [ ]:
# Test the recommendation system
recommended_books = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(recommended_books)


In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):


  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()